In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from symbol import Symbol
from index import Index
from portifolio import Portifolio
import seaborn as sns
from sklearn.preprocessing import scale
from scipy import stats


sns.set_style("dark")

%matplotlib inline

In [ ]:
sym = Symbol()
ind = Index()
port = Portifolio()
ret = sym.get_symbol_data('returns')

In [ ]:
def refd(returns, risk_free_rate=0.075):
    rf = pd.DataFrame(index=pd.date_range(start=ret.index[0],end=ret.index[-1]), columns=['daily', 'weekly', 'monthly', 'quarterly', 'annual'])
    rf['annual'] = risk_free_rate
    years = returns.index.year
    dai = returns.resample('A').count()
    rf['daily'] = risk_free_rate/dai
    dai = returns.resample('W').count().resample('A').count()
    rf['weekly'] = risk_free_rate/dai
    dai = returns.resample('M').count().resample('A').count()
    rf['monthly'] = risk_free_rate/dai
    dai = returns.resample('Q').count().resample('A').count()
    rf['quarterly'] = risk_free_rate/dai
    ind = rf.index.intersection(returns.index)
    rf = rf.bfill().ffill()
    rf = rf[rf.index.isin(ind)]
    return rf.dropna()
risk_free_rate = refd(ret)
# (ret.infy.resample('W').sum() - risk_free_rate.weekly).sum()


In [ ]:
from helpers import rename_columns
rff = pd.read_excel('rf.xlsx')
rename_columns(rff)
rff = rff.set_index('date')/100
rf = pd.DataFrame(index=pd.date_range(start=ret.index[0],end='2017-12-31'), columns=['daily', 'weekly', 'monthly', 'quarterly', 'annual'])
returns = ret.copy()
# dai = returns.resample('M').count().max(axis=1)
rf['monthly'] = rff.price.resample('M').mean()
dai = returns.resample('A').count().max(axis=1)
rf['annual'] = rf.resample('A').monthly.mean()
dai = returns.resample('Q').count().max(axis=1)
rf['quarterly'] = rf.resample('Q').monthly.mean()
# dai = returns.resample('W').count().max(axis=1)
rf['weekly'] = rf.resample('W').monthly.mean()
dai = returns.resample('D').count().max(axis=1).resample('A').sum()
rf['daily'] = rf.resample('D').monthly.mean()/dai
rf['monthly'] = (rf['monthly']/12)
rf['weekly'] = (rf['weekly']/52)
rf['quarterly'] = (rf['quarterly']/4)

rf = rf.bfill().ffill()
# ind = rf.index.intersection(ret.index)
# rf = rf.bfill().ffill()
rf = rf[rf.index.isin(ind)]
rf['2016']


In [ ]:
from helpers import rename_columns
rff = pd.read_excel('rfda.xlsx')
rename_columns(rff)
rff.date = pd.to_datetime(rff.date)
rff = rff.set_index('date')/100
rfda = pd.DataFrame(index=pd.date_range(start=rff.index[-1],end=rff.index[0]), columns=['daily', 'weekly', 'monthly', 'quarterly', 'annual'])
rfda['daily'] = rff
rfda = rfda.bfill().ffill()
rfda['annual'] = rfda.resample('A').daily.mean()
rfda['quarterly'] = rfda.resample('Q').daily.mean()
rfda['weekly'] = rfda.resample('W').daily.mean()
rfda['monthly'] = rfda.resample('D').daily.mean()
dai = returns.resample('D').count().max(axis=1).resample('A').sum()
rfda['monthly'] = (rfda['monthly']/12)
rfda['weekly'] = (rfda['weekly']/52)
rfda['quarterly'] = (rfda['quarterly']/4)
rfda['daily'] = rfda.resample('D').daily.mean()/dai.resample('D').mean().bfill()[:'2016']
rfda.daily = rfda['daily'].fillna(rff['2017'].resample('D').mean().price/252)

rfda = rfda[rfda.index.isin(rf.index)]
rfda = rfda.bfill().ffill()
rf[rf.index.isin(rfda.index)] = rfda
rf.columns = ['D', 'W', 'M', 'Q', 'A']
rf.plot()
rf.to_hdf('constants.h5', 'risk_free_rate')